In [57]:
import io
import requests
import PyPDF2

def split_pdf_pages(pdf_data):
    pages = []
    reader = PyPDF2.PdfFileReader(io.BytesIO(pdf_data))

    for page_number in range(reader.getNumPages()):
        if reader.getPage(page_number):
            pages.append(reader.getPage(page_number))

    return pages

In [63]:
urls = ["https://file.notion.so/f/s/32916f5d-5a33-4bdb-8270-1f4a640d95de/Programmabegroting_2020_Valkenswaard.pdf?id=d7a82aab-2a0d-40c8-945a-534c482a7324&table=block&spaceId=ca274338-dfb2-4f65-823d-2acde6195fea&expirationTimestamp=1684398564708&signature=ynvhZqaeL7EE_OG-tV_BVPVZD9QDeG0AvkghgMTywX8&downloadName=Programmabegroting+2020+Valkenswaard.pdf"]

In [64]:
import urllib.parse

def extract_title(url):
    parsed_url = urllib.parse.urlparse(url)
    query_params = urllib.parse.parse_qs(parsed_url.query)
    download_name = query_params.get("downloadName", [None])[0]
    return download_name

In [65]:
keywords = ["bouw", "groen", "groene"]

In [66]:
def count_keywords_in_string(string):
    keyword_count = 0
    for keyword in keywords:
        keyword_count += string.lower().count(keyword.lower())
    return keyword_count

In [70]:
for url in urls:
    response = requests.get(url)
    pdf_data = response.content
    pdf_pages = split_pdf_pages(pdf_data)
    for page_number, page in enumerate(pdf_pages):
        contents = page.extract_text()
        if contents =="":
            continue
        if not any(keyword in contents for keyword in keywords):
            continue
        if count_keywords_in_string(contents) <= 5:
            continue
        print("page number: ", page_number)
        print(page.extract_text())
        print("\n")

page number:  11
Begroting 2020-2023 
12 
 kunnen we ons overgebleven woningbouwprogramma snel  realiseren. De nadruk ligt daarbij op de 
(her)ontwikkeling van terreinen binnen onze bebouwd e kom. Gedurende deze raadsperiode 
actualiseren, realiseren en monitoren we ons woning bouwprogramma voor de toekomst. Zodat we 
ook meerjarig onze strategische doelstellingen kunn en behalen. 
 
Strategie gericht op jongeren en jonge gezinnen 
Vergrijzing en ontgroening zijn trends die de bevol kingsomvang nu en in de toekomst beïnvloeden. 
Deze ontwikkelingen zetten de bestaande sociale str ucturen onder druk. Om ervoor te zorgen dat 
het hoge voorzieningenniveau van Valkenswaard in de  toekomst behouden kan blijven, is het 
namelijk van belang dat het aantal inwoners minimaa l op hetzelfde peil blijft en het liefste groeit. 
Daarom zetten we in op het behouden en aantrekken v an nieuwe inwoners, met name jongeren en 
jonge gezinnen, Zo ontstaat er meer (sociaal en fin ancieel) draagvlak voor voo